In [71]:
#KBO 투수 기록 스크래핑 2022.8.16 jonghee5park@gmail.com
#사용 모듈: requests, bs4, selenium, pandas, time
import selenium
from selenium import webdriver
from selenium.webdriver.support.ui import Select
import time
import requests
from bs4 import BeautifulSoup
import pandas as pd
from pandas import DataFrame

#함수 선언
def List2Arr(head, data): #스크래핑 결과를 2차원 배열로 변환
    parent_arr = []
    base = len(head)
    row = int(len(data) / base);
    for i in range(base):
        node_arr = []
        for j in range(row):
            offset = i + (base * j)
            node_arr.append(data[offset])
        parent_arr.append(node_arr)
    return parent_arr

#Select team list
team_l =['OB','LT','SS','WO','HH','HT','KT','LG','NC','SK']  

driver = webdriver.Chrome()


#기본기록 스크래핑
driver.get("https://www.koreabaseball.com/Record/Player/PitcherBasic/Basic1.aspx")

result_html = driver.page_source
soup = BeautifulSoup(result_html, 'lxml')
ths = soup.find_all('th')

#table head 추출
head = []
for th in ths:
    head.append(th.get_text())

basic_df = pd.DataFrame(columns = head)

for team in team_l: #팀별 반복

    dropdown = Select(driver.find_element_by_xpath('//*[@id="cphContents_cphContents_cphContents_ddlTeam_ddlTeam"]'))
    dropdown.select_by_value(team)

    time.sleep(1)

    result_html = driver.page_source
    soup = BeautifulSoup(result_html, 'lxml')
    tds = soup.find_all('td')

    data = []
    for i, td in enumerate(tds):
        try:
            data.append(int(td.get_text()))
        except:
            try:
                data.append(float(td.get_text()))
            except:
                data.append(td.get_text())

    temp_df = pd.DataFrame(List2Arr(head, data)).T
    temp_df.columns = head
    basic_df = pd.concat([basic_df, temp_df], ignore_index=True)

    

#세부기록 스크래핑
driver.get("https://www.koreabaseball.com/Record/Player/PitcherBasic/Detail1.aspx")

result_html = driver.page_source
soup = BeautifulSoup(result_html, 'lxml')
ths = soup.find_all('th')

head = []
for th in ths:
    head.append(th.get_text())

detail_df = pd.DataFrame(columns = head)

for team in team_l: #팀별 반복

    dropdown = Select(driver.find_element_by_xpath('//*[@id="cphContents_cphContents_cphContents_ddlTeam_ddlTeam"]'))
    dropdown.select_by_value(team)

    time.sleep(1)

    result_html = driver.page_source
    soup = BeautifulSoup(result_html, 'lxml')
    tds = soup.find_all('td')

    data = []
    for i, td in enumerate(tds):
        try:
            data.append(int(td.get_text()))
        except:
            try:
                data.append(float(td.get_text()))
            except:
                data.append(td.get_text())

    temp_df = pd.DataFrame(List2Arr(head, data)).T
    temp_df.columns = head
    detail_df = pd.concat([detail_df, temp_df], ignore_index=True)
    
merged_df = pd.merge(basic_df, detail_df)
    
driver.close()
driver.quit()

merged_df

,순위,선수명,팀명,ERA,G,W,L,SV,HLD,WPCT,...,GS,Wgs,Wgr,GF,SVO,TS,GDP,GO,AO,GO/AO
0,1,정철원,두산,3.02,41,3,2,1,13,0.6,...,0,0,3,5,14,0,5,61,51,1.2
1,2,스탁,두산,3.03,22,9,6,0,0,0.6,...,22,9,0,0,0,0,22,154,99,1.56
2,3,박정수,두산,3.2,13,1,1,0,0,0.5,...,0,0,1,2,1,0,2,20,18,1.11
3,4,최원준,두산,3.42,20,7,8,0,0,0.467,...,20,7,0,0,0,0,7,89,160,0.56
4,5,브랜든,두산,3.6,2,1,0,0,0,1.0,...,2,1,0,0,0,0,0,20,5,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
249,23,정성곤,SSG,10.13,2,0,0,0,0,-,...,0,0,0,0,0,0,0,3,2,1.5
250,25,김주온,SSG,11.25,5,0,0,0,0,-,...,0,0,0,4,0,0,0,6,2,3.0
251,26,신재영,SSG,12.46,4,0,0,0,0,-,...,0,0,0,2,0,0,1,5,3,1.67
252,27,한두솔,SSG,16.88,8,0,0,0,0,-,...,0,0,0,3,0,0,0,10,5,2.0
